In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
sys.path.append("..") # Adds higher directory to python modules path.
import brevis
from brevis import branches
from brevis import evaluate


In [9]:
# K = 10
# def loss_fn(annealing_rate=1, momentum=1, decay=1, global_loss=False):
#     #create a wrapper function that returns a function
#     temperature = 1
#     Classes = 10
#     keras_kl = tf.keras.losses.KLDivergence()
#     annealing_rate = annealing_rate
#     momentum_rate = momentum
#     decay_rate = decay
#     def cross_entropy_evidence(labels, outputs): 
#         softmax = tf.nn.softmax(outputs)
#         # activated_outputs =tf.keras.activations.sigmoid(softmax)
#         evidence = softplus_evidence(outputs)
#         alpha = evidence + 1
#         S = tf.reduce_sum(alpha, axis=1, keepdims=True) 
#         E = alpha - 1
#         m = alpha / S
#         A = tf.reduce_sum((labels-m)**2, axis=1, keepdims=True) 
#         B = tf.reduce_sum(alpha*(S-alpha)/(S*S*(S+1)), axis=1, keepdims=True) 

#         annealing_coef = tf.minimum(1.0,tf.cast(annealing_rate,tf.float32))
# #         annealing_coef = 1
#         alp = E*(1-labels) + 1 
#         # print("alp", alp)
#         C =  annealing_coef * KL(alp)
#         # C = keras_kl(labels,evidence)
#         loss = tf.keras.losses.categorical_crossentropy(labels, softmax)
#         pred = tf.argmax(outputs,1)
#         truth = tf.argmax(labels,1)
#         match = tf.reshape(tf.cast(tf.equal(pred, truth), tf.float32),(-1,1))
#         return loss + C
#         # return (A + B) + C
#     return  cross_entropy_evidence
loss_fn = brevis.utils.evidence_crossentropy()

In [10]:
brevisModel = brevis.BranchModel(name="../models/brevis_alex_Bflat.hdf5")
brevisModel.compile(loss=["CategoricalCrossentropy",loss_fn,loss_fn,loss_fn], optimizer=tf.optimizers.SGD(learning_rate=0.001, momentum=0.9), preset="")

_map_graph_network

preset: Other
_map_graph_network

preset: Other


In [11]:
train_ds, test_ds, validation_ds = brevis.dataset.prepare.dataset(tf.keras.datasets.cifar10.load_data(),32,5000,22500,(227,227),include_targets=True)

augment Dataset
targetsis : True
adding targets to inputs
trainSize 45000
testSize 10000


In [12]:
brevisModel.evaluate(test_ds)

312/312 [==============================] - 22s 71ms/step - loss: 3.4742 - dense_2_loss: 0.6684 - branch_softmax_loss: 1.1531 - branch_softmax_1_loss: 0.8553 - branch_softmax_2_loss: 0.7974 - dense_2_accuracy: 0.8117 - branch_softmax_accuracy: 0.6117 - branch_softmax_1_accuracy: 0.7096 - branch_softmax_2_accuracy: 0.8062 - branch_softmax_evidence: 0.0066 - branch_softmax_mean_ev_succ: 0.0813 - branch_softmax_mean_ev_fail: 0.0419 - branch_softmax_1_evidence: 0.0100 - branch_softmax_1_mean_ev_succ: 0.1280 - branch_softmax_1_mean_ev_fail: 0.0309 - branch_softmax_2_evidence: 0.2013 - branch_softmax_2_mean_ev_succ: 2.4431 - branch_softmax_2_mean_ev_fail: 0.2079


[3.4741899967193604,
 0.6683692336082458,
 1.153076410293579,
 0.8553193211555481,
 0.797425389289856,
 0.8116987347602844,
 0.6116786599159241,
 0.7096354365348816,
 0.8061898946762085,
 0.006618714891374111,
 0.0812828317284584,
 0.04191126674413681,
 0.0099885119125247,
 0.12796013057231903,
 0.03091173991560936,
 0.2012537270784378,
 2.443101644515991,
 0.20794060826301575]

In [50]:
test_set = brevis.dataset.prepare.test_set(tf.keras.datasets.cifar10.load_data(),1,(227,227),include_targets=True)

stopping_point =None
num_outputs = len(brevisModel.outputs)
print(num_outputs)
predictions = []
labels = []
pClass = []
predictions=[]
pEvidence = []
pUncertainty=[]
pOverlap=[]

Outputs = pd.DataFrame()
pAcc=[]
for i in range(num_outputs):
    pClass.append([])
    predictions.append([])
    pEvidence.append([])
    pUncertainty.append([])
    pAcc.append([])
    pOverlap.append([])
for i, (x,y) in enumerate(test_set):
    if stopping_point and i > stopping_point:
        break
    print("prediction: {} of {}".format(i,len(test_set)),end='\r')
    if True: 
        result = brevisModel.test_on_batch(x,y)
        if i < 1:
            print(result)
#             print(result)
        for j in range(num_outputs):
            # print("output",j)
            pClass[j].append(tf.argmax(y[0]).numpy())
#                 print("class",pClass[j][i])
            pAcc[j].append(result[j+(num_outputs)+1])  
#                 print("acc",pAcc[j][i])
            if j ==0:
                pEvidence[j].append(0)
            else:
#                     print("evid Number",((num_outputs * 2)+1), " ", ((j-1)*3))
                pEvidence[j].append(result[((num_outputs * 2) + 1)+((j-1)*3)])
#                 print("evid",pEvidence[j][i])

            pOverlap[j].append(pAcc[0][i] - pAcc[j][i])
#                 print("overlap",pOverlap[j][i])
    
    '''
    overlap
    if zero, both match, if else they don't match
    TT 1-1 =0
    TF 1-0 =1

    FT 0-1 = -1
    FF 0-0 =0

    '''
Outputs=[]
for j in range(num_outputs):
    Predictions = pd.DataFrame({"label":pClass[j],"evidence":pEvidence[j],"Acc":pAcc[j], "overlap":pOverlap[j]})
    Outputs.append(Predictions)
print(len(Outputs[1]))

targetsis : True
adding targets to inputs
4
[1.0044565200805664, 0.24729721248149872, 0.31445106863975525, 0.2363983392715454, 0.206309974193573, 1.0, 1.0, 1.0, 1.0, 0.002966057974845171, 0.029660580679774284, 0.0, 0.009362570010125637, 0.09362570196390152, 0.0, 0.04273209348320961, 0.4273209273815155, 0.0]
10000ction: 9999 of 10000


In [52]:
# print(Outputs)
# for i in range(5):
# print(Outputs[1])
print(Outputs[0].groupby('Acc').count())
print(Outputs[1].groupby('Acc').count())
print(Outputs[2].groupby('Acc').count())
print(Outputs[3].groupby('Acc').count())
# print(Outputs[4].groupby('Acc').count())




     label  evidence  overlap
Acc                          
0.0   1883      1883     1883
1.0   8117      8117     8117
     label  evidence  overlap
Acc                          
0.0   3886      3886     3886
1.0   6114      6114     6114
     label  evidence  overlap
Acc                          
0.0   2906      2906     2906
1.0   7094      7094     7094
     label  evidence  overlap
Acc                          
0.0   1939      1939     1939
1.0   8061      8061     8061


In [70]:

for i, Predictions in enumerate(Outputs):
    mean = Predictions.loc[(Predictions['Acc'] == False)].groupby("Acc")["evidence"].mean().iloc[0]
    # print("mean",mean)
    std = Predictions.loc[(Predictions['Acc'] == False)].groupby("Acc")["evidence"].std().iloc[0]
    # print("std",std)
    threshold = mean + std
    
    print(threshold)
    Accuracy = len(Predictions.loc[(Predictions["evidence"] >= threshold) &  (Predictions['Acc'] == True)]) / len(Predictions.loc[(Predictions["evidence"] >= threshold)])
    Accepted = Predictions.loc[(Predictions["evidence"] >= threshold)]
    Rejected = Predictions.loc[(Predictions["evidence"] < threshold)]
    print("Accuracy", Accuracy)
    print("Accepted", len(Accepted))
    print("Rejected", len(Rejected))
    


0.0
Accuracy 0.8117
Accepted 10000
Rejected 0
0.01205414801339648
Accuracy 0.7542441390460792
Accepted 1237
Rejected 8763
0.00878253051121819
Accuracy 0.8949089210649229
Accepted 2141
Rejected 7859
0.07322921868124949
Accuracy 0.9727157360406091
Accepted 4728
Rejected 5272


In [27]:
brevisModel.evaluate(test_set)

10000/10000 [==============================] - 758s 76ms/step - loss: 3.4740 - dense_2_loss: 0.6682 - branch_softmax_loss: 1.1530 - branch_softmax_1_loss: 0.8557 - branch_softmax_2_loss: 0.7971 - dense_2_accuracy: 0.8117 - branch_softmax_accuracy: 0.6114 - branch_softmax_1_accuracy: 0.7094 - branch_softmax_2_accuracy: 0.8061 - branch_softmax_evidence: 0.0066 - branch_softmax_mean_ev_succ: 0.0499 - branch_softmax_mean_ev_fail: 0.0163 - branch_softmax_1_evidence: 0.0100 - branch_softmax_1_mean_ev_succ: 0.0908 - branch_softmax_1_mean_ev_fail: 0.0091 - branch_softmax_2_evidence: 0.2012 - branch_softmax_2_mean_ev_succ: 1.9725 - branch_softmax_2_mean_ev_fail: 0.0393


[3.4740288257598877,
 0.6682007312774658,
 1.1530203819274902,
 0.8556945323944092,
 0.7971065640449524,
 0.8116999864578247,
 0.6114000082015991,
 0.7093999981880188,
 0.8061000108718872,
 0.0066134571097791195,
 0.04986374452710152,
 0.016270719468593597,
 0.009982521645724773,
 0.090774305164814,
 0.009051105938851833,
 0.20118005573749542,
 1.9724615812301636,
 0.039336394518613815]

In [13]:
test_set = brevis.dataset.prepare.test_set(tf.keras.datasets.cifar10.load_data(),1,(227,227),include_targets=True)
thresholds = brevisModel.evaluate_branch_thresholds(test_set)
thresholds

targetsis : True
adding targets to inputs
outputs 4
[1.0044565200805664, 0.24729721248149872, 0.31445106863975525, 0.2363983392715454, 0.206309974193573, 1.0, 1.0, 1.0, 1.0, 0.002966057974845171, 0.029660580679774284, 0.0, 0.009362570010125637, 0.09362570196390152, 0.0, 0.04273209348320961, 0.4273209273815155, 0.0]
[0.42847132682800293, 0.03387385606765747, 0.038467489182949066, 0.06505706906318665, 0.29107290506362915, 1.0, 1.0, 1.0, 1.0, 0.003420871216803789, 0.03420871123671532, 0.0, 0.0012019594432786107, 0.012019594199955463, 0.0, 0.014800223521888256, 0.1480022370815277, 0.0]
Doneiction: 9999 of 10000
      label  evidence  Acc  overlap
0         3         0  1.0      0.0
1         8         0  1.0      0.0
2         8         0  0.0      0.0
3         0         0  1.0      0.0
4         6         0  1.0      0.0
...     ...       ...  ...      ...
9995      8         0  1.0      0.0
9996      3         0  0.0      0.0
9997      5         0  1.0      0.0
9998      1         0  1.

{'dense_2/Softmax:0': 0.0,
 'branch_softmax/MatMul:0': -0.003680132983169616,
 'branch_softmax_1/MatMul:0': -0.0025532800891417675,
 'branch_softmax_2/MatMul:0': -0.03265532713135014}

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
sys.path.append("..") # Adds higher directory to python modules path.
import brevis
from brevis import branches
from brevis import evaluate
loss_fn = brevis.utils.evidence_crossentropy()
brevisModel = brevis.BranchModel(name="../models/brevis_alex_Bflat.hdf5")
# brevisModel.summary()
brevisModel.compile(loss=["CategoricalCrossentropy",loss_fn,loss_fn,loss_fn], optimizer=tf.optimizers.SGD(learning_rate=0.001, momentum=0.9), preset="")
test_set = brevis.dataset.prepare.test_set(tf.keras.datasets.cifar10.load_data(),1,(227,227),include_targets=True)

# print(thresholds)
thresholds={}
thresholds['branch_softmax/MatMul:0']= 0.01205414801339648
thresholds['branch_softmax_1/MatMul:0']=    0.00878253051121819      
thresholds['branch_softmax_2/MatMul:0']=   0.07322921868124949       
for layer in brevisModel.layers:
    if issubclass(type(layer),brevis.branches.branch.BranchEndpoint): ## check if the layer is a branch endpoint or subclass
        for key in thresholds.keys():
            if layer.name+'/' in key:
                print (layer.name)
                layer.threshold = thresholds[key]
        print(layer.threshold)


_map_graph_network

preset: Other
_map_graph_network

preset: Other
targetsis : True
adding targets to inputs
branch_softmax
0.01205414801339648
branch_softmax_1
0.00878253051121819
branch_softmax_2
0.07322921868124949


In [2]:
brevisModel.set_branchExits(True)

branch_softmax :  True
branch_softmax_1 :  True
branch_softmax_2 :  True


In [3]:
# brevisModel._map_graph_network(brevisModel.inputs, brevisModel.outputs)

_map_graph_network
conv2d_1 :  18   input_2 :  19
batch_normalization :  17   conv2d_1 :  18
max_pooling2d :  16   batch_normalization :  17
conv2d_2 :  15   max_pooling2d :  16
batch_normalization_1 :  14   conv2d_2 :  15
max_pooling2d_1 :  13   batch_normalization_1 :  14
conv2d_3 :  12   max_pooling2d_1 :  13
batch_normalization_2 :  11   conv2d_3 :  12
conv2d_4 :  10   batch_normalization_2 :  11
batch_normalization_3 :  9   conv2d_4 :  10
conv2d_5 :  8   batch_normalization_3 :  9
batch_normalization_4 :  7   conv2d_5 :  8
max_pooling2d_2 :  6   batch_normalization_4 :  7
flatten :  5   max_pooling2d_2 :  6
dense :  4   flatten :  5
dropout :  3   dense :  4
dense_1 :  2   dropout :  3
dropout_1 :  1   dense_1 :  2
dense_2 :  0   dropout_1 :  1
branch_flatten :  15   max_pooling2d :  16
branch124 :  14   branch_flatten :  15
branch64 :  13   branch124 :  14
branch_softmax :  12   branch64 :  13
branch_softmax :  12   targets :  1
branch_flatten_1 :  12   max_pooling2d_1 :  13
bran

x = test_set.take(1)

In [9]:
brevisModel.set_branchExits(True,['branch_softmax_1'])


branch_softmax_1 :  True


In [10]:
# %%timeit
# print(x)
for i, data in enumerate(x):
    results = brevisModel.predict(data[0])
    print(results)
    # print(tf.argmax(results[0]))

input_2
conv2d_1
batch_normalization
max_pooling2d
branch_flatten
conv2d_2
branch124
batch_normalization_1
max_pooling2d_1
branch64
branch_flatten_1
branch_softmax
conv2d_3
branch124_1
batch_normalization_2
branch64_1
conv2d_4
branch_softmax_1
[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32), [array([[-10.740568 , -11.373923 ,  -8.842008 ,  -2.5738642,  -7.490037 ,
         -4.0288725,  -6.646391 ,  -9.881697 ,  -8.233668 ,  -8.719537 ]],
      dtype=float32)], array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)]


In [24]:
brevisModel.set_branchExits(False)


branch_softmax :  False
branch_softmax_1 :  False
branch_softmax_2 :  False


In [25]:
%%timeit
# print(x)
for i, data in enumerate(x):
    results = brevisModel.predict(data[0])
    # print(results)
    # print(tf.argmax(results[0]))

6.09 s ± 271 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
np.argmax(x[1])

3

In [22]:
for i in results:
    print(np.argmax(i))

0
3
0
0
